# Second stage of the OSIOSN project

### Creating and training baseline model

In [6]:
import os
import numpy as np
import torch
from torch import nn, cat
from torch.nn import Conv1d, ConvTranspose1d, MaxPool1d, Dropout, ReLU, Sequential, Sigmoid

In [4]:
DATASET_PATH = "./dataset"

In [5]:
x_train = np.load(os.path.join(DATASET_PATH, "x_train.npy"))
y_train = np.load(os.path.join(DATASET_PATH, "x_train.npy"))

x_test = np.load(os.path.join(DATASET_PATH, "x_train.npy"))
y_test = np.load(os.path.join(DATASET_PATH, "x_train.npy"))

x_val = np.load(os.path.join(DATASET_PATH, "x_train.npy"))
y_val = np.load(os.path.join(DATASET_PATH, "x_train.npy"))

In [ ]:
num_epochs = 20
batch_size = 5
lr = 1e-3

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Nauka sieci realizowana będzie na:',device)

In [36]:
# data shape [batch_size, channels, samples]

class Conv1DBlock(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size):
    super(Conv1DBlock, self).__init__()
    self.conv1d = Conv1d(in_channels, out_channels, kernel_size, padding="same")
    self.relu = ReLU(inplace=True)

  def forward(self, x):
    x = self.conv1d(x)
    x = self.relu(x)
    return x

class DoubleConv1DBlock(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size):
    super(DoubleConv1DBlock, self).__init__()
    self.double_conv = Sequential(
      Conv1DBlock(in_channels, out_channels, kernel_size),
      Conv1DBlock(out_channels, out_channels, kernel_size)
    )

  def forward(self, x):
    return self.double_conv(x)


class DownSample1DBlock(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size, dropout=0.3):
    super(DownSample1DBlock, self).__init__()
    self.double_conv = DoubleConv1DBlock(in_channels, out_channels, kernel_size)
    self.maxpool_1d = MaxPool1d(kernel_size=2)
    self.dropout = Dropout(dropout)

  def forward(self, x):
    x = self.double_conv(x)
    pool = self.maxpool_1d(x)
    return x, self.dropout(pool)


class UpSample1DBlock(nn.Module):
  def __init__(self, in_channels, out_channels, kernel_size, dropout=0.3):
    super(UpSample1DBlock, self).__init__()
    self.conv1d_transpose = ConvTranspose1d(in_channels, out_channels, kernel_size=2, stride=2)
    self.double_conv = DoubleConv1DBlock(in_channels, out_channels, kernel_size)
    self.dropout = Dropout(dropout)

  def forward(self, x, conv_features):
    x = self.conv1d_transpose(x)
    print(x.size())
    print(conv_features.size())
    x = cat([x, conv_features], dim=1)
    x = self.dropout(x)
    return self.double_conv(x)

In [41]:
class RPNet(nn.Module):
  def __init__(self, in_channels, out_channels):
    super(RPNet, self).__init__()
    self.d1 = DownSample1DBlock(in_channels, 16, 3)
    self.d2 = DownSample1DBlock(16, 32, 3)
    self.d3 = DownSample1DBlock(32, 64, 3)

    self.bottleneck = DoubleConv1DBlock(64, 128, 3)
    
    self.u1 = UpSample1DBlock(128, 64, 3)
    self.u2 = UpSample1DBlock(64, 32, 3)
    self.u3 = UpSample1DBlock(32, 16, 3)

    self.output = Conv1d(16, out_channels, kernel_size=1)
    self.sigmoid = Sigmoid()

  def forward(self, x):
    f1, p1 = self.d1(x)
    f2, p2 = self.d2(p1)
    f3, p3 = self.d3(p2)
    
    bottleneck = self.bottleneck(p3)

    u1 = self.u1(bottleneck, f3)
    u2 = self.u2(u1, f2)
    u3 = self.u3(u2, f1)

    output = self.output(u3)
    return self.sigmoid(output)

In [42]:
rpnet = RPNet(in_channels=1, out_channels=1)
input_data = torch.randn(1, 1, int(10 * 100))  # Example input data
output_data = rpnet(input_data)
print(output_data)

torch.Size([1, 64, 250])
torch.Size([1, 64, 250])
torch.Size([1, 32, 500])
torch.Size([1, 32, 500])
torch.Size([1, 16, 1000])
torch.Size([1, 16, 1000])
tensor([[[0.5420, 0.5432, 0.5414, 0.5428, 0.5422, 0.5442, 0.5432, 0.5445,
          0.5443, 0.5461, 0.5435, 0.5434, 0.5458, 0.5431, 0.5424, 0.5466,
          0.5440, 0.5453, 0.5395, 0.5432, 0.5428, 0.5398, 0.5428, 0.5415,
          0.5433, 0.5442, 0.5405, 0.5450, 0.5405, 0.5457, 0.5428, 0.5433,
          0.5431, 0.5463, 0.5426, 0.5436, 0.5401, 0.5431, 0.5402, 0.5469,
          0.5427, 0.5465, 0.5418, 0.5437, 0.5446, 0.5413, 0.5414, 0.5414,
          0.5450, 0.5467, 0.5442, 0.5422, 0.5430, 0.5427, 0.5440, 0.5408,
          0.5407, 0.5428, 0.5440, 0.5442, 0.5443, 0.5418, 0.5445, 0.5451,
          0.5414, 0.5427, 0.5406, 0.5401, 0.5398, 0.5409, 0.5438, 0.5416,
          0.5476, 0.5418, 0.5431, 0.5396, 0.5446, 0.5424, 0.5390, 0.5455,
          0.5436, 0.5442, 0.5414, 0.5464, 0.5405, 0.5414, 0.5436, 0.5468,
          0.5445, 0.5414, 0.5437, 